# Detekcja krawędzi

## Cel ćwiczenia

- Zapoznanie z metodami detekcji krawędzi:
    - Sobel, Prewitt, Roberts - przypomnienie,
    - Laplasjan z Gaussa (LoG – ang. Laplacian of Gaussian),
    - Canny.

Detekcja krawędzi przez wiele lat była podstawą algorytmów segmentacji.
Krawędzie wykrywane są najczęściej z wykorzystaniem pierwszej (gradient) i drugiej (Laplasjan) pochodnej przestrzennej.
Wykorzystanie obu metod zaprezentowane zostało w ćwiczeniu *Przetwarzanie wstępne. Filtracja kontekstowa*.

W niniejszym ćwiczeniu poznane detektory krawędzi zostaną porównane z bardziej zaawansowanymi: Laplasjan z funkcji Gaussa (LoG), Zero Crossing i Canny.

## Laplasjan z Gaussa (LoG)

Funkcja Gaussa:<br>
\begin{equation}
h(r) = e^{\frac{-r^2}{2 \sigma^2}}
\end{equation}<br>
gdzie:
- $r^2 = x^2 + y^2$
- $\sigma$ to odchylenie standardowe.

Działanie filtracji Gaussowskiej zostało przedstawione w ćwiczeniu "Przetwarzanie wstępne". W jej wyniku następuje rozmazanie obrazu.
Laplasjan tej funkcji dany jest wzorem:

\begin{equation}
\nabla^2 h(r) = \frac{r^2 - 2\sigma^2}{\sigma^4} e^{-\frac{r^2}{2\sigma^2}}
\end{equation}

Funkcję (z oczywistych powodów) nazywamy Laplasjan z Gaussa (LoG).
Ponieważ druga pochodna jest operacją liniową, konwolucja obrazu z $\nabla^2 h(r)$ daje taki sam efekt jak zastosowanie filtracji Gaussa na obrazie, a następnie obliczenie Laplasjanu z wyniku.
Lokalizacja krawędzi polega na znalezieniu miejsca, gdzie po filtracji LoG następuje zmiana znaku.

1. Wczytaj obraz *house.png*.
2. Wykonaj rozmycie Gaussowskie obrazu wejściowego.
W tym celu wykorzysaj funkcję `cv2.GaussianBlur(img, kSize, sigma)`.
Pierwszy argument jest obrazem wejśćiowym.
Drugi jest rozmiarem filtru (podanym w nawiasach okrągłych, np. *(3, 3)*).
Trzecim argumentem jest odchylenie standardowe. Wartość jest dobrana automatycznie, jeśli zosanie podana wartość `0` (będą równe rozmiarowi).
3. Oblicz laplasjan obrazu rozmytego.
W tym celu wykorzysaj funkcję `cv2.Laplacian(img, ddepth)`.
Pierszym argumentem jest obraz wejściowy.
Drugim argumentem jest typ danych wejściowych. Użyj `cv2.CV_32F`.
4. Wyznacz miejsca zmiany znaku.
Zaimplementuj funkcję `crossing(LoG, thr)`:
    - Najpierw stwórz tablicę, do której zostanie zapisany wynik.
    Jej rozmiar jest taki sam jak przetwarzanego obrazu.
    - Następnie wykonaj pętle po obrazie (bez ramki jednopikselowej).
    W każdej iteracji stwórz otoczenie o rozmiarze $3 \times 3$.
    Dla otoczenia oblicz wartość maksymalną i minimalną.
    - Jeśli wartości te mają przeciwne znaki, to do danego miejsca tablicy przypisz wartość:
        - jeśli piksel wejściowy > 0, to dodaj do niego wartość bezwzględną minimum.
        - jeśli piksel wejściowy < 0, to do jego wartości bezwzględnej dodaj maksimum.
    - Zmień zakres wykonanej tablicy do $<0, 255>$.
    - Wykonaj progowanie tablicy. Próg jest argumentem wejściowym.
    - Przeskaluj dane binarne do wartości `[0, 255]`.
    - Wykonaj konwersję do typu *uint8*.
    - Wykonaj rozmycie medianowe wyniku.
    Wykorzystaj funkcję `cv2.medianBlur(img, kSize)`.
    Pierwszym argumentem jest obraz wejśćiowy, a drugim rozmiar filtra.
    - Zwróć wyznaczoną tablicę.
5. Wyświetl obraz wynikowy.
6. Dobierz parametry (rozmiar filtru Gaussa, odchylenie standardowe, próg binaryzacji) tak, by widoczne były kontury domu, ale nie dachówki.

In [ ]:
import cv2
from matplotlib import pyplot as plt
import numpy as np
import math
import os
import requests

url = 'https://raw.githubusercontent.com/vision-agh/poc_sw/master/09_Canny/'

fileNames = ["dom.png"]
for fileName in fileNames:
  if not os.path.exists(fileName):
      r = requests.get(url + fileName, allow_redirects=True)
      open(fileName, 'wb').write(r.content)



In [ ]:
image = cv2.imread('dom.png')
image = cv2.cvtColor(image, cv2.COLOR_RGB2GRAY)
image = cv2.GaussianBlur(image, (7,7), 1.6)
image = cv2.Laplacian(image, cv2.CV_32F)

image.shape

plt.imshow(image, 'gray', vmin=0, vmax=255)

In [ ]:
def crossing(LoG, thr):
  image = LoG
  result_shape_x, result_shape_y = image.shape
  result = np.zeros((result_shape_x-2,result_shape_y-2))
  for i in range(1,result_shape_y-1):
    for j in  range(1,result_shape_x-1):
      max_val = np.max(image[max(j-1,0):min(j+2, result_shape_x),max(i-1,0):min(i+2, result_shape_y)])
      min_val = np.min(image[max(j-1,0):min(j+2, result_shape_x),max(i-1,0):min(i+2, result_shape_y)])
      if min_val<0 and max_val>0:
        if(image[j-1,i-1]>0): result[j-1,i-1]=image[j,i]+np.abs(min_val)
        elif(image[j-1,i-1]<0): result[j-1,i-1]=np.abs(image[j,i])+max_val
      else:
          result[j-1,i-1]=image[j,i]
  result = cv2.normalize(result, None, 0, 256, cv2.NORM_MINMAX)
  result = result.astype(np.uint8)
  result = cv2.threshold(result, thr, 255, cv2.THRESH_BINARY)[1]
  result * 255
  result = cv2.medianBlur(result, 3)
  return result


new_img = crossing(image,120)
plt.imshow(new_img, 'gray', vmin=0, vmax=255)

## Algorytm Canny'ego

> Algorytm Canny'ego to często wykorzystywana metoda detekcji krawędzi.
> Zaproponowana została w 1986r. przez Johna F. Cannego.
> Przy jego projektowaniu założono trzy cele:
> - niska liczba błędów - algorytm powinien znajdywać wszystkie krawędzie oraz generować jak najmniej fałszywych detekcji,
> - punkty krawędziowe powinny być poprawnie lokalizowane - wykryte punkty powinny być jak najbardziej zbliżone do rzeczywistych,
> - krawędzie o szerokości 1 piksela - algorytm powinien zwrócić jeden punkt dla każdej rzeczywistej krawędzi.

Zaimplementuj algorytm detekcji krawędziCanny'ego:
1. W pierwszym kroku obraz przefiltruj dwuwymiarowym filtrem Gaussa.
2. Następnie oblicz gradient pionowy i poziomy ($g_x $ i $g_y$).
Jedną z metod jest filtracja Sobela.
3. Dalej oblicz amplitudę:
$M(x,y)  = \sqrt{g_x^2+g_y^2}$ oraz kąt:
$\alpha(x,y) = arctan(\frac{g_y}{g_x})$.
Do obliczenia kąta wykorzystaj funkcję `np.arctan2(x1, x2)`.
Wynik jest w radianach.
4. W kolejnym etapie wykonaj kwantyzację kątów gradientu.
Kąty od $-180^\circ$ do $180^\circ$ można podzielić na 8 przedziałów:
[$-22.5^\circ, 22.5^\circ$], [$22.5^\circ, 67.5^\circ$],
[$67.5^\circ, 112.5^\circ$], [$112.5^\circ, 157.5^\circ$],
[$157.5^\circ, -157.5^\circ$], [$-157.5^\circ, -112.5^\circ$],
[$-112.5^\circ, -67.5^\circ$], [$-67.5^\circ, -22.5^\circ$].
Przy czym należy rozpatrywać tylko 4 kierunki:
    - pionowy ($d_1$),
    - poziomy ($d_2$),
    - skośny lewy ($d_3$),
    - skośny prawy ($d_4$).
5. Dalej przeprowadź eliminację pikseli, które nie mają wartości maksymalnej (ang. *nonmaximal suppresion*).
Celem tej operacji jest redukcja szerokości krawędzi do rozmiaru 1 piksela.
Algorytm przebiega następująco.
W rozpatrywanym otoczeniu o rozmiarze $3 \times 3$:
    - określ do którego przedziału należy kierunek gradientu piksela centralnego ($d_1, d_2, d_3, d_4$).
    - przeanalizuje sąsiadów leżących na tym kierunku.
Jeśli choć jeden z nich ma amplitudę większą niż piksel centralny, to należy uznać, że nie jest lokalnym maksimum i do wyniku przypisać $g_N(x,y) = 0$.
W przeciwnym przypadku $g_N(x,y) = M(x,y)$.
Przez $g_N$ rozumiemy obraz detekcji lokalnych maksimów.
Zaimplementuj funkcję `nonmax`.
Pierwszym argementem jest macierz kierunków (po kwantyzacji).
Drugim argumentem jest macierz amplitudy.
6. Ostatnią operacją jest binaryzacja obrazu $g_N$.
Stosuje się tutaj tzw. binaryzację z histerezą.
Wykorzystuje się w niej dwa progi: $T_L$ i $T_H$, przy czym $T_L < T_H$.
Canny zaproponował, aby stosunek progu wyższego do niższego był jak 3 lub 2 do 1.
Rezultaty binaryzacji można opisać jako:<br>
$g_{NH}(x,y) = g_N(x,y) \geq TH $<br>
$g_{NL}(x,y) = TH > g_N(x,y) \geq TL $<br>
Można powiedzieć, że na obrazie $g_{NH}$ są "pewne" krawędzie.
Natomiast na $g_{NL}$ "potencjalne".
Często krawędzie "pewne" nie są ciągłe.
Dlatego wykorzystuje się obraz $g_{NL}$ w następującej procedurze:
    - Stwórz stos zawierający wszystkie piksele zaznaczone na obrazie $g_{NH}$.
W tym celu wykorzystaj listę współrzędnych `[row, col]`.
Do pobrania elementu z początku służy metoda `list.pop()`.
Do dodania elementu na koniec listy służy metoda `list.append(new)`.
    - Stwórz obraz, który będzie zawierał informację czy dany piksel został już odwiedzony.
    - Stwórz obraz, któy zawierać będzie wynikowe krawędzie.
Jej rozmiar jest równy rozmiarowi obrazu.
    - Wykonaj pętlę, która będzie pobierać elementy z listy, dopóki ta nie będzie pusta.
W tym celu najlepiej sprawdzi się pętla `while`.
    - W każdej iteracji pobierz element ze stosu.
    - Sprawdź czy dany element został już odwiedzony.
    - Jeśli nie został, to:
        - Oznacz go jako odwiedzony,
        - Oznacz piksel jako krawędź w wyniku,
        - Sprawdź otoczenie piksela w obrazie $g_{NL}$,
        - Dodaj do stosu współrzędne otoczenia, które zawierają krawędź (potencjalną).
        Można to wykoanać np. pętlą po stworzonym otoczeniu.
7. Wyświetl obraz oryginalny, obraz $g_{NH}$ oraz obraz wynikowy.

Pomocnicze obrazy $g_{NH}$ i $g_{NL}$ zostały wprowadzone dla uproszczenia opisu.
Algorytm można zaimplementować wbardziej "zwarty" sposób.

Na podstawie powyższego opisu zaimplementuj algorytm Cannego.

In [ ]:
image = cv2.imread('dom.png')
image = cv2.cvtColor(image, cv2.COLOR_RGB2GRAY)
image = cv2.GaussianBlur(image, (5,5),1.6)

plt.imshow(image, 'gray', vmin=0, vmax=255)
r1 = np.array([[-1,0,1],[-2,0,2],[-1,0,1]])
r2 = np.array([[1,2,1],[0,0,0],[-1,-2,-1]])
gx = cv2.filter2D(image,-1, r1)
gy = cv2.filter2D(image,-1,r2)
M = np.hypot(gx, gy)


angle = np.arctan2(gy, gx)
angle = np.degrees(angle)
angle[angle < 0] += 180

H, W = image.shape
Z = np.zeros((H, W))
plt.imshow(image, 'gray', vmin=0, vmax=255)

In [ ]:

for y in range(1, H-1):
  for x in range(1, W-1):

    if -22.5<=angle[y, x]<22.5 or 157.5<=angle[y, x]<=180:
      p = M[y, x+1]
      q = M[y, x-1]
    elif 22.5<=angle[y, x]<67.5:
      p = M[y+1, x-1]
      q = M[y-1, x+1]
    elif 67.5<=angle[y, x]<112.5:
      p = M[y-1,x]
      q = M[y+1, x]
    elif 112.5<=angle[y, x]<157.5:
      p = M[y-1, x-1]
      q = M[y+1, x+1]
    if M[y,x]>=p and M[y,x]>=q:
      Z[y,x] = M[y, x]
    else:
      Z[y, x] = 0


max_threshold = 63
Z_GNL = np.logical_and(31 <= Z, Z <= max_threshold).astype(int)

Z_GNH = (Z >= max_threshold).astype(int)
stack = []
[stack.append([y,x]) for x in range(0, W-2)  for y in range(0, H-2) if Z_GNH[y, x] == 1]
pixel_observer = np.array([[0 for x in range(0, W-2)] for y in range(0, H-2)])
new_Z = np.zeros((H,W), dtype=np.int32)

while stack:
  A = stack.pop()
  if(pixel_observer[A[0],A[1]]==0):
      pixel_observer[A[0],A[1]] = 1
      new_Z[A[0],A[1]] = 1
      surroundings = Z_GNL[max(A[0]-1,0):min(H-2, A[0]+2), max(A[1]-1,0):min(W-2, A[1]+2)]
      for i, y in enumerate(surroundings):
        for j, x in enumerate(y):
          if(x==1):
            stack.append([A[0]+i-1, A[1]-1+j])



fx, ax = plt.subplots(1,2)
fx.set_size_inches(20, 10)
ax[0].imshow(Z_GNH, 'gray', vmin=0, vmax=1)
ax[1].imshow(new_Z, 'gray', vmin=0, vmax=1)

In [ ]:
import cv2
import numpy as np



image = cv2.imread('dom.png')
image = cv2.cvtColor(image, cv2.COLOR_RGB2GRAY)
image = cv2.GaussianBlur(image, (5,5),1.6)

r1 = np.array([[-1,0,1],[-2,0,2],[-1,0,1]])
r2 = np.array([[1,2,1],[0,0,0],[-1,-2,-1]])
gx = cv2.filter2D(image,-1, r1)
gy = cv2.filter2D(image,-1,r2)
M = np.hypot(gx, gy)

angle = np.arctan2(gy, gx)
angle = np.degrees(angle)
angle[angle < 0] += 180



# Kwantyzacja kątów i eliminacja pikseli niebędących lokalnymi maksimami
def non_max_suppression(M, angle):
    H, W = M.shape
    Z = np.zeros((H, W), dtype=np.float32)
    angle = angle % 180

    for i in range(1, H-1):
        for j in range(1, W-1):
            try:

               # Kąty i ich kwantyzacja
                if (0 <= angle[i, j] < 22.5) or (157.5 <= angle[i, j] <= 180):
                    q = M[i, j + 1]
                    r = M[i, j - 1]
                elif 22.5 <= angle[i, j] < 67.5:
                    q = M[i + 1, j - 1]
                    r = M[i - 1, j + 1]
                elif 67.5 <= angle[i, j] < 112.5:
                    q = M[i + 1, j]
                    r = M[i - 1, j]
                elif 112.5 <= angle[i, j] < 157.5:
                    q = M[i - 1, j - 1]
                    r = M[i + 1, j + 1]

                if (M[i, j] >= q) and (M[i, j] >= r):
                    Z[i, j] = M[i, j]
                else:
                    Z[i, j] = 0

            except IndexError as e:
                pass

    return Z

Z = non_max_suppression(M, angle)

# Binaryzacja z histerezą
def thresholding(Z):
    highThreshold = 63
    lowThreshold = 31
    print(highThreshold, lowThreshold)
    H, W = Z.shape
    res = np.zeros((H, W), dtype=np.int32)

    # Silne i słabe piksele
    strong_i, strong_j = np.where(Z >= highThreshold)
    zeros_i, zeros_j = np.where(Z < lowThreshold)

    weak_i, weak_j = np.where((Z <= highThreshold) & (Z >= lowThreshold))

    res[strong_i, strong_j] = 255
    res[weak_i, weak_j] = 75

    return res

res = thresholding(Z)

# Śledzenie krawędzi przez histerezę
def hysteresis(res):
    H, W = res.shape
    for i in range(1, H-1):
        for j in range(1, W-1):
            if res[i, j] == 75:
                if 255 in [res[i+1, j-1], res[i+1, j], res[i+1, j+1], res[i, j-1], res[i, j+1], res[i-1, j-1], res[i-1, j], res[i-1, j+1]]:
                    res[i, j] = 255
                else:
                    res[i, j] = 0
    return res

edges = hysteresis(res)
fx, ax = plt.subplots(1,2)
fx.set_size_inches(20, 10)
ax[0].imshow(image, 'gray', vmin=0, vmax=1)
ax[1].imshow(edges, 'gray', vmin=0, vmax=1)



## Algorytm Canny'ego - OpenCV

1. Wykonaj dektekcję krawędzi metodą Canny'ego wykorzystując funkcję `cv2.Canny`.
    - Pierwszym argumentem funkcji jest obraz wejściowy.
    - Drugim argumentem jest mniejszy próg.
    - Trzecim argumentem jest większy próg.
    - Czwarty argument to tablica, do której wpisany zostanie wynik.
    Można zwrócić go przez wartość i podać wartość `None`.
    - Piąty argument to rozmiar operatora Sobela (w naszym przypadku 3).
    - Szósty argument to rodzaj używanej normy.
    0 oznacza normę $L_1$, 1 oznacza normę $L_2$. Użyj $L_2$.
2. Wynik wyświetl i porównaj z własną implementacją.

In [ ]:
image = cv2.imread('dom.png')
image = cv2.cvtColor(image, cv2.COLOR_RGB2GRAY)
image = cv2.GaussianBlur(image, (5,5),1.6)
image_canny = cv2.Canny(image, 31, 63, None, 3, 1)
plt.imshow(image_canny, 'gray', vmin=0, vmax=1)